In [1]:
import cv2
img=cv2.imread("Face.png", 1)
# cv2.imshow("Nitesh", img)
# cv2.waitKey()
# cv2.destroyAllWindows()
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces=face_cascade.detectMultiScale(gray_img, scaleFactor=1.05, minNeighbors=5)
for x, y, w, h in faces:
    img=cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 3)
resized=cv2.resize(img, (int(img.shape[1]/2), int(img.shape[0]/2)))
cv2.imshow("Face", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import cv2, time
video=cv2.VideoCapture(0)
check, frame=video.read()
print(check)
#print(frame)
time.sleep(3)
cv2.imshow('Capturing', frame)
cv2.waitKey(0)
video.release()
cv2.destroyAllWindows()

True


In [3]:
import cv2, time
video=cv2.VideoCapture(0)
a=1
while True:
    a=a+1
    check, frame=video.read()
    #print(frame)
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Capturing', frame)
    key=cv2.waitKey(1)
    if key == ord('q'):
        break;
print(a)
video.release()
cv2.destroyAllWindows()

244


In [1]:
import cv2, time
import pandas as pd
from datetime import datetime

first_frame=None
status_list=[None, None]
time=[]
df=pd.DataFrame(columns=['Start', 'End'])
video=cv2.VideoCapture(0)
while True:
    check, frame=video.read()
    #status at the begining of recording is zero as the object is not visible
    status=0
    #print(frame)
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray, (21, 21), 0)
    if first_frame is None:
        first_frame=gray
        #time.append(datetime.now())
        continue
    #calculate the difference b/w first frame and other frame
    delta_frame=cv2.absdiff(first_frame, gray)
    #Provides a threshold value such that it will convert the difference value with less than 30 to black, otherwise white
    thresh_delta=cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_delta=cv2.dilate(thresh_delta, None, iterations=0)
    #Define contour area, basically add the border
    (cnts,_)=cv2.findContours(thresh_delta.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in cnts:
        #removes noises and shadows.
        #Basically it will keep only that part white, which has area greater than 1000 pixels.
        if cv2.contourArea(contour)<1000:
            continue
        #Change in status when the object is being detected
        status=1
        #Create a rectangular box around object of frame
        (x, y, w, h)=cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 3)
    status_list.append(status)
    status_list=status_list[-2:]
    if status_list[-1]==1 and status_list[-2]==0:
        time.append(datetime.now())
    if status_list[-1]==0 and status_list[-2]==1:
        time.append(datetime.now())
    cv2.imshow('frame',frame)
#     cv2.imshow('gray',gray)
#     cv2.imshow('delta_frame',delta_frame)
#     cv2.imshow('thresh_delta',thresh_delta)
    key=cv2.waitKey(1)
    if key == ord('q'):
        if status == 1:
            time.append(datetime.now())
        break
print(time)
#print('\n')
for i in range(0, len(time), 2):
    df=df.append({'Start': time[i], 'End':time[i+1]}, ignore_index=True)
df.to_csv('Times.csv')
video.release()
cv2.destroyAllWindows()

[datetime.datetime(2022, 11, 11, 16, 24, 16, 183765), datetime.datetime(2022, 11, 11, 16, 25, 3, 562947)]


C:\Users\Dell Vostro 3491\AppData\Local\Temp\ipykernel_13360\3648336085.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'Start': time[i], 'End':time[i+1]}, ignore_index=True)
